<a href="https://colab.research.google.com/github/CinthiaS/spell-checker-pt/blob/main/Spell_checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets
!pip install happytransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from datasets import load_dataset
import pandas as pd
import re
import random
from happytransformer import HappyTextToText
import pickle
import unicodedata

In [6]:
with open('/content/drive/My Drive/Colab Notebooks/Natural Language Processing with Transformers/Spell checker/data/portuguese.pkl', 'rb') as f:

  docs = pickle.load(f)

In [7]:
sentences = []
for i in docs.items():
  for key, values in i[1].items():
    sentences.append(values['document'])

## Assin2

In [ ]:
dataset = load_dataset("assin2")

ValueError: ignored

In [ ]:
sentences = []
for i in dataset['train']:
  sentences.append(i['premise']+ '.')
  sentences.append(i['hypothesis'] + ".")

In [8]:
df = pd.DataFrame()
df['target'] = sentences
df['input'] = sentences

In [10]:
def put_punctuation(text):
  return text + "."

In [9]:
def remove_punctuation(text):
  return re.sub(r'[^\w\s]','',text)

In [11]:
def remove_random_character(phrase, n_remove=10):
    for num in random.sample(range(0, len(phrase)), n_remove):
        phrase = phrase[:num] + '' + phrase[num + 1:]
    return phrase

In [12]:
def define_task(text):
  return "grammar: " + text

In [13]:
def remove_accents(text):
  return ''.join((c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn'))

In [14]:
df['input'] = df['input'].str.lower()

In [15]:
df['input'] = df['input'].apply(remove_punctuation)

In [16]:

df['input'] = df['input'].apply(remove_random_character)

In [17]:
df['input'] = df['input'].apply(define_task)

In [18]:
df['input'] = df['input'].apply(remove_accents)

In [ ]:
df.to_csv("/content/drive/My Drive/Colab Notebooks/spell_checker.csv")

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['input'], df['target'], test_size=0.1, random_state=42)

In [20]:
train = pd.DataFrame([X_train, y_train], index= ['input', 'target']).T
train

,input,target
53667,grammar: a embolia pulmonar macica um coagulo ...,A embolia pulmonar maciça (um coágulo sanguíne...
58887,grammar: com um martelo pregue um prego na par...,"Com um martelo, pregue um prego na parede em u..."
38532,grammar: ficar pensando se esta ou nao na hora...,Ficar pensando se está ou não na hora de envia...
70284,grammar: nao existeuma frequencia padrao com ...,Não existe uma frequência padrão com a qual a...
53395,grammar: concenrese nas coisas que o interessa...,Concentre-se nas coisas que o interessam para ...
...,...,...
6265,grammar: ele e representado por um icone verme...,Ele é representado por um ícone vermelho com u...
54886,grammar: e melhor aproveitar ao maximo o ervic...,É melhor aproveitar ao máximo o serviço enquan...
76820,grammar: segureas sob agua corrent e sfregue a...,Segure-as sob água corrente e esfregue a casca...
860,grammar: quando voce ouvir preparar evante sua...,"Quando você ouvir “preparar”, levante suas cos..."


In [21]:
test = pd.DataFrame([X_test, y_test], index= ['input', 'target']).T
test

,input,target
32730,grammar: imagine que voce quira saber o valor ...,Imagine que você queira saber o valor da veloc...
58068,grammar: falar sobre a separacao nao e facil m...,"Falar sobre a separação não é fácil, mas é nec..."
80423,grammar: para fazelo acesse o endereco httpwor...,"Para fazê-lo, acesse o endereço http://wordtoj..."
25953,grammar: esse metodo funcionamelhor nos cabelo...,Esse método funciona melhor nos cabelos molhad...
75608,grammar: o processo e totalmente imprevisivel ...,O processo é totalmente imprevisível. Se não f...
...,...,...
76427,grammar: nao va logo pensando que todos os fu...,. Não vá logo pensando que todos os funcionári...
71938,grammar: ler em voz alta e um otimo exercicio ...,"Ler em voz alta é um ótimo exercício, ainda ma..."
628,grammar: um comprador inteligente sabe que ele...,Um comprador inteligente sabe que ele tem a va...
72787,grammar: assoe o narz regularmente para remove...,Assoe o nariz regularmente para remover o muco...


In [22]:
train.to_csv("/content/drive/My Drive/Colab Notebooks/Natural Language Processing with Transformers/Spell checker/train.csv", index=False)

In [23]:
test.to_csv("/content/drive/My Drive/Colab Notebooks/Natural Language Processing with Transformers/Spell checker/test.csv", index=False)

## Fine Tuning T5 model

In [3]:
happy_tt = HappyTextToText("PTT5", "unicamp-dl/ptt5-base-portuguese-vocab")

In [4]:
train_path = '/content/drive/My Drive/Colab Notebooks/Natural Language Processing with Transformers/Spell checker/test.csv'

In [28]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-f55e6c9d-44c4-1e63-f603-8d325f753175)


In [29]:
from happytransformer import TTTrainArgs

args = TTTrainArgs(batch_size=1)
happy_tt.train(train_path, args=args)

  0%|          | 0/1 [00:00<?, ?it/s]

PyTorch: setting up devices
***** Running training *****
  Num examples = 8166
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 24498
  Number of trainable parameters = 222903552


OutOfMemoryError: ignored

In [ ]:
happy_tt.save("/content/drive/My Drive/Colab Notebooks/Natural Language Processing with Transformers/Spell checker/model/")

Configuration saved in /content/drive/My Drive/Colab Notebooks/Natural Language Processing with Transformers/Spell checker/model/config.json
Model weights saved in /content/drive/My Drive/Colab Notebooks/Natural Language Processing with Transformers/Spell checker/model/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/Colab Notebooks/Natural Language Processing with Transformers/Spell checker/model/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/Colab Notebooks/Natural Language Processing with Transformers/Spell checker/model/special_tokens_map.json
Copy vocab file to /content/drive/My Drive/Colab Notebooks/Natural Language Processing with Transformers/Spell checker/model/spiece.model


In [ ]:
from happytransformer import TTSettings

beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=20)

In [ ]:
example_1 = "grammar: esa sentenca possuiestos escritos incoetamente"
result_1 = happy_tt.generate_text(example_1, args=beam_settings)
print(result_1.text)

A pessoa possui textos escritos inquietamente.
